In [4]:
from myHeap import *
#numpy is needed to better manage arrays and indexing of matrices
import numpy as np

The implementation of Dijkstra's algorithm with a Min-Heap requires a slightly modification of the Min-Heap and the nodes in the graph.

The problem is that Dijkstra's algorithm requires to, extract the root from the heap, find its neighbours and update the distance from the source of them. To do this 2 operation we have to work both on the graph and on the heap so at each step each node has to know "where it is" on the graph and on the heap.

The solution is based on the design choice to give each node a member called `Heap_idx` which stores the position of the node on the Heap. Then the Heap is a `Heap_of_Nodes` in which every time a node is moved on the heap updates the `Heap_idx` member of the nodes, so the coherence is mantained through the algorithm and data structures.


In [5]:
class node:
    """
    Implementation of node member with also representation of usefull members
    """
    def __init__(self,val):
        self.val = val
        self.color = None
        self.d = None
        self.pred = None
        self.index = None
        self.Heap_idx = None
        self.graph_idx = None
       
    def __str__(self):
        return "(Val %s Heap_idx %s d %s)" % (str(self.val),str(self.Heap_idx),str(self.d))
    def __repr__(self):
        return (__class__.__name__).join(self.__str__())

class Graph:
    """
    General purpose graph implementation
    """
    import numpy as np
    def __init__ (self,E,V):
        self.E = np.array(E)                
        self.V = np.array([node(val) for val in V])
        self.tot_nodes = len(self.V)
        for i in range(len(self.V)):
            self.V[i].index = i
        self.COLORS = ['white','grey','black']
        self.Q = None
        self.BYPASSED_nodes = []
        
        
        
    def _node_total_order(self,n1,n2):
        return n1.d < n2.d    
        
    def __str__(self):
        return "Edges (adj matrix): \n %s \n Nodes \n %s" % (str(self.E), str(self.V))
       
    def _Adj_idx(self,node_idx):
        return [i for i in range(self.tot_nodes) if self.E[node_idx,i] != 0]
    
    def _Adj_node(self,node):
        node_idx = node.index
        return self._Adj_idx(node_idx)
 
    def Adj(self,node):
        return self.V[self._Adj_node(node)]
    def _pred(self):
        return [v.pred for v in self.V]
    def _d(self):
        return [v.d for v in self.V]

class SimpleGraph(Graph):
    """
    Simple non weighted graph, inherits from graph,
    made only for practicing, not requested, nor well commented
    """
    
        
    def _BFS_set(self,node,color,d,pred):
        
        node.color = color
        node.d = d
        if pred:
            node.pred = pred.index
        else:
            node.pred = None
        
    def _BFS_init(self,source):
        for v in self.V:
            self._BFS_set(v,'white', np.infty, None)
        self._BFS_set(source,'grey',0,source)
        
        self.Q = Heap([source],self._node_total_order)
        
    def BFS(self,source_idx):
        source = self.V[source_idx]
        self._BFS_init(source)
        while self.Q.Size != 0:
            u = self.Q.ExtractRoot()
            
            for v in self.Adj(u):
                if v.color == 'white':
                    self._BFS_set(v,'grey', u.d + 1, u)
                    self.Q.Insert(v)
            u.color = 'black'
        
        return self._pred(), self._d()
    
class WeightedGrah(Graph):
    """
    Implementation of Weighted graph, inherits from Graph
    """
    def _SSSP_init(self):
        """
        Initialization step of SSSP
        """
        for v in self.V:
            v.d = np.infty
            v.pred = None
            
    def _UPDATE_dist(self, v, dist):
        """
        Updates the distance of the node passed and
        asks the Heap to push up the node on the Heap,
        to make sure the Heap property is fixed after the update
        """
        v.d = dist
        self.Q._push_up(v.Heap_idx)
        
        
        
    def _relax(self,u,v,w):
        """
        Dijkstra's algorithm step to update the distance of the node's neighbours
        Setting also the predecessor of the node if condition is fullfilled
        """
        if u.d + w < v.d:
            self._UPDATE_dist(v, u.d + w)
            v.pred = u.index
        
            
    def _Adj_w(self,node):
        """
        Returns the adjiacent nodes of a given node and given
        weights in an iterable, relies on _Adj_node defined in parent class
        """
        n_list = self._Adj_node(node)
        return zip(self.V[n_list], self.E[node.index,n_list])
    
    def _set_Heap_idx(self):
        """
        Sets all the Heap_idx of the nodes in the Heap
        """
        if self.Q:
            for i in range(self.Q.Size):
                self.Q.A[i].Heap_idx = i
        
    def DIJKSTRA_node(self, source):
        """
        Implementation of Dijkstra's algorithm using `Heap_of_nodes` defined in 
        `myHeap.py` using as total oreder `_node_total_order` defined in parent class
        """
        #initialize all the nodes
        self._SSSP_init()
        source.d = 0
        #build the heap
        self.Q = Heap_of_nodes(self.V, self._node_total_order)
        #set all Heap_idx to current values
        self._set_Heap_idx()
        
        while self.Q.Size != 0:
            #perform extraction of a node
            u = self.Q.ExtractRoot()
            u = self.V[u.index]
            
            for v,w in  self._Adj_w(u):
                #relaxation dijkstra step
                #v -> neighobour node, w -> weight of the edge connecting them
                self._relax(u,v,w)
                
        #returning all the properties of the nodes calculated
        return self._pred(), self._d()
    
    
    def DIJKSTRA(self, source_idx):
        return self.DIJKSTRA_node(self.V[source_idx])
    
    def add_shortcut(self, node_idx):
        """
        Calculates new edges of the graph corresponding to shortucuts on the graph
        The idea is that given a node "k" the algorithm takes all the edges entering and exiting k
        Let assume i has an edge  into k and o has an edge arriving from k
        If there is no edge between i and o a new edge is created in E[i,o] = E[i,k] + E[k,o]
        If there is already an edge the minimum is taken among the current one and the 
        calculated through the shortcut
        """
        self.BYPASSED_nodes.append(node_idx)
        node_in = np.array([ i for i in range(len(self.V)) if self.E[i,node_idx] != 0])
        node_out = np.array([ j for j in range(len(self.V)) if self.E[node_idx,j] != 0])
        
        for idx in node_in:
            for jdx in node_out:
                if self.E[idx,jdx] > 0:
                    self.E[idx,jdx] = min(self.E[idx,jdx],self.E[idx,node_idx]+self.E[node_idx,jdx])
                else:
                    self.E[idx,jdx] = self.E[idx,node_idx]+self.E[node_idx,jdx]

 

In [8]:
#test graph taken from slides, to test Dijkstra algorithm result
w = np.array([
    [0,1,5,0,0,0],
    [0,0,0,0,0,15],
    [0,0,0,2,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,3],
    [0,0,0,0,0,0]
])  

vertx = ['a' ,'b','c','d','e','f','g']
v = [i for i in range(1,7)]
print(v)
W_graph = WeightedGrah(w,v)

[1, 2, 3, 4, 5, 6]


In [9]:
W_graph.DIJKSTRA(0)
#first element are predecessors, second distances

([None, 0, 0, 2, 3, 4], [0, 1, 5, 7, 8, 11])

In [14]:
W_graph.add_shortcut(2)
#adding shortcut to bypass node 2
#checking what happens if we apply simple dijkstra to it 
#checking also what happens at the edge matrix

In [12]:
W_graph.DIJKSTRA(0)

([None, 0, 0, 0, 3, 4], [0, 1, 5, 7, 8, 11])

In [13]:
W_graph.E

array([[ 0,  1,  5,  7,  0,  0],
       [ 0,  0,  0,  0,  0, 15],
       [ 0,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  0]])